REQUIRED LIBRARY INSTALLATION

In [ ]:
pip install -r requirements.txt


LIBRARY

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from mpltern.ternary import TernaryAxes
import os
import re
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor
import shap
import pickle
from matplotlib.collections import PolyCollection



LOAD DATA FILE

In [ ]:
file_path = 'DataSets.xlsx'  # Update this with the correct path if needed
data = pd.read_excel(file_path, sheet_name='DataSet')

# Inspect data to ensure it’s loaded correctly
print(data.head())
data.describe()

DATA STATISTICS

In [ ]:

# Set output directory
output_dir = "parameter_plots"
os.makedirs(output_dir, exist_ok=True)

# Initialize list to store statistics
stats_summary = []

# Filter numerical columns
numerical_columns = data.select_dtypes(include=[np.number]).columns

# Compute statistics
for column in numerical_columns:
    col_data = data[column].dropna()
    mean_val = col_data.mean()
    median_val = col_data.median()
    std_val = col_data.std()
    q1 = np.percentile(col_data, 25)
    q3 = np.percentile(col_data, 75)
    iqr = q3 - q1
    min_val = col_data.min()
    max_val = col_data.max()

    stats_summary.append({
        'Column': column,
        'Mean': mean_val,
        'Median': median_val,
        'Std Dev': std_val,
        'Q1 (25%)': q1,
        'Q3 (75%)': q3,
        'IQR': iqr,
        'Min': min_val,
        'Max': max_val
    })

# Create DataFrame and save to CSV
stats_df = pd.DataFrame(stats_summary)
stats_file_path = os.path.join(output_dir, "summary_statistics_from_excel.csv")
stats_df.to_csv(stats_file_path, index=False)

print(f"\nSummary statistics saved to {stats_file_path}")


In [ ]:


# Create output directory
output_dir = "parameter_plots"
os.makedirs(output_dir, exist_ok=True)

# Define column groups
ultimate_columns = ['C (wt%)', 'H (wt%)', 'N (wt%)', 'O (wt%)', 'S (wt%)']
proximate_columns = ['Volatile (wt%)', 'Fixed carbon (wt%)', 'Ash (wt%)']
process_columns = ['Sample (g)','Temperature (°C)', 'Heating rate (°Cmin-1)', 'Reaction time (min)']
yield_columns = ['Syngas yield (%)', 'Char yield (%)', 'Pyrolysis oil yield (%)']
gas_columns = ['CO2 (mol%)', 'CH4 (mol%)', 'CO (mol%)', 'H2 (mol%)']

# Define colors for each group
colors = {
    'Ultimate': 'skyblue',
    'Proximate': 'salmon',
    'Process': 'lightgreen',
    'Yield': 'orchid',
    'Gas': 'gold'
}

# Function to sanitize filenames
def sanitize_filename(name):
    # Replace problematic characters with underscores
    invalid_chars = ['<', '>', ':', '"', '/', '\\', '|', '?', '*', '°', '%', '(', ')', ' ']
    filename = name
    for char in invalid_chars:
        filename = filename.replace(char, '_')
    # Remove any duplicate underscores
    while '__' in filename:
        filename = filename.replace('__', '_')
    return filename

# Function to create half violin plot with box plot
def half_violin_box_plot(data, column, color, category):
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Calculate statistics for consistent y positioning
    min_val = data[column].min() 
    max_val = data[column].max()
    range_val = max_val - min_val
    
    # Add padding to y-axis
    y_min = min_val - range_val * 0.1
    y_max = max_val + range_val * 0.1
    
    # Define positions for the violin and box plots
    violin_pos = 0.3
    box_pos = 0.7
    
    # Create violin plot
    violin_parts = ax.violinplot(data[column], positions=[violin_pos], 
                           vert=True, widths=0.3, showmeans=False, 
                           showextrema=False, showmedians=False)
    
    # Customize violin plot - keep only the left half
    for pc in violin_parts['bodies']:
        verts = pc.get_paths()[0].vertices
        # Only keep the left half of the violin
        verts[:, 0] = np.clip(verts[:, 0], violin_pos - 0.15, violin_pos)
        pc.set_facecolor(color)
        pc.set_edgecolor('black')
        pc.set_alpha(0.8)
    
    # Add box plot on the right
    box_parts = ax.boxplot(data[column], positions=[box_pos], 
                     vert=True, widths=0.3, patch_artist=True, 
                     showcaps=True, showfliers=True)
    
    # Customize box plot
    for component in ['boxes', 'whiskers', 'caps', 'medians']:
        for patch in box_parts[component]:
            patch.set_color('black')
    
    for patch in box_parts['boxes']:
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    # Add reference lines for mean and median
    mean_val = data[column].mean()
    median_val = data[column].median()
    
    ax.axhline(mean_val, color='red', linestyle='-', linewidth=1.5, alpha=0.7, label=f'Mean: {mean_val:.2f}')
    ax.axhline(median_val, color='blue', linestyle='--', linewidth=1.5, alpha=0.7, label=f'Median: {median_val:.2f}')
    
    # Calculate summary statistics
    std_val = data[column].std()
    q1 = np.percentile(data[column], 25)
    q3 = np.percentile(data[column], 75)
    iqr = q3 - q1
    
    # Add statistics as text
    stats_text = (
        f"Mean: {mean_val:.2f}\n"
        f"Median: {median_val:.2f}\n"
        f"Std Dev: {std_val:.2f}\n"
        f"IQR: {iqr:.2f}\n"
        f"Range: {min_val:.2f} - {max_val:.2f}"
    )
    
    # Position text in the upper right
    ax.text(0.95, 0.95, stats_text, transform=ax.transAxes,
            verticalalignment='top', horizontalalignment='right',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Set labels and title
    ax.set_title(f"{category} - {column}", fontsize=14)
    ax.set_ylabel(column, fontsize=12)
    
    # Remove x-ticks since they're not meaningful
    ax.set_xticks([])
    
    # Set y-axis limits with padding
    ax.set_ylim(y_min, y_max)
    
    # Add grid for readability
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Add legend
    ax.legend(loc='upper left')
    
    # Create safe filename
    safe_category = sanitize_filename(category)
    safe_column = sanitize_filename(column)
    filename = f"half_violin_box_{safe_category}_{safe_column}.png"
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, filename), dpi=300)
    plt.close()
    
    return filename

# Process all columns
column_groups = {
    'Ultimate': ultimate_columns,
    'Proximate': proximate_columns,
    'Process': process_columns,
    'Yield': yield_columns,
    'Gas': gas_columns
}

# List to store created filenames
created_files = []

# Create individual plots for each column
for category, columns in column_groups.items():
    for column in columns:
        if column in data.columns:  # Assumes 'data' dataframe is defined elsewhere
            try:
                filename = half_violin_box_plot(data, column, colors[category], category)
                created_files.append(filename)
                print(f"Created half violin plot with box plot for {column}")
            except Exception as e:
                print(f"Error creating plot for {column}: {e}")

# Print summary
print(f"\nAnalysis complete. Created {len(created_files)} half violin plots with box plots.")
print(f"All plots saved to {output_dir} directory.")

PEARSON PLOT


In [ ]:


# Define the features - properly formatted as a list of strings
features = [
    'C (wt%)', 'H (wt%)', 'N (wt%)', 'O (wt%)', 'S (wt%)', 
    'Volatile (wt%)', 'Fixed carbon (wt%)', 'Ash (wt%)',
    'Sample (g)', 'Temperature (°C)', 'Heating rate (°Cmin-1)',
    'Reaction time (min)', 'Syngas yield (%)',
    'CH4 (mol%)', 'H2 (mol%)', 'Char yield (%)',
    'Pyrolysis oil yield (%)'
]

# Create output directory
output_dir = "correlation_plots"
os.makedirs(output_dir, exist_ok=True)

# Filter dataset to include only the selected features
# Remove the 'Sample' column if it's non-numeric to avoid correlation errors
numeric_features = [f for f in features if f != 'Sample']
data_filtered = data[numeric_features]

# Compute correlation matrix
correlation_matrix = data_filtered.corr(method='pearson')

# Create a mask for the upper triangle to avoid redundancy
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Plot heatmap with improved styling
plt.figure(figsize=(16, 14))
sns.set(font_scale=1.2)  # Increase font size for better readability

# Create the heatmap with better styling
heatmap = sns.heatmap(
    correlation_matrix, 
    annot=True,               # Show correlation values
    cmap="coolwarm",          # Use a diverging colormap
    fmt=".2f",                # Format numbers to 2 decimal places
    linewidths=0.5,           # Add grid lines                # Apply mask to show only lower triangle
    cbar_kws={"shrink": 0.8}, # Adjust colorbar size
    annot_kws={"size": 10}    # Adjust annotation text size
)

# Improve the visualization
plt.title("Pearson Correlation Heatmap of Pyrolysis Parameters", fontsize=18, pad=20)
plt.xticks(rotation=45, ha='right', fontsize=12)  # Rotate x labels for better visibility
plt.yticks(rotation=0, fontsize=12)               # Keep y labels horizontal
plt.tight_layout()                                # Adjust layout to fit everything

# Save the figure
plt.savefig(os.path.join(output_dir, "pearson_correlation_heatmap.png"), dpi=300, bbox_inches='tight')

# Show the plot
plt.show()

print(f"Correlation heatmap saved to {os.path.join(output_dir, 'pearson_correlation_heatmap.png')}")

Ternary plot

In [ ]:
# Define input variables and output
syngas_yield = data['Syngas yield (%)']
char_yield = data['Char yield (%)']
pyrolysis_oil_yield = data['Pyrolysis oil yield (%)']
temperature = data['Temperature (°C)']  # Output variable (this will be the color)

# Normalize the ternary inputs to sum to 1
total = syngas_yield + char_yield + pyrolysis_oil_yield
syngas_norm = syngas_yield / total
char_norm = char_yield / total
oil_norm = pyrolysis_oil_yield / total

# Create figure
fig = plt.figure(figsize=(10.8, 4.8))
fig.subplots_adjust(left=0.075, right=0.85, wspace=0.3)

# Set color scale range
vmin = temperature.min()
vmax = temperature.max()
levels = np.linspace(vmin, vmax, 7)

# First subplot with flat shading
ax1 = fig.add_subplot(1, 2, 1, projection='ternary')
cs1 = ax1.tripcolor(syngas_norm, char_norm, oil_norm, temperature, 
                   shading='flat', vmin=vmin, vmax=vmax)

ax1.set_tlabel('Syngas Yield')
ax1.set_llabel('Char Yield')
ax1.set_rlabel('Pyrolysis Oil Yield')

# Add colorbar to first subplot
cax1 = ax1.inset_axes([1.05, 0.1, 0.05, 0.9], transform=ax1.transAxes)
colorbar1 = fig.colorbar(cs1, cax=cax1)
colorbar1.set_label('Temperature (°C)', rotation=270, va='baseline')

# Save the figure
plt.savefig(os.path.join(output_dir, "Syngas Yield.png"), dpi=300, bbox_inches='tight')

plt.show()

print(f"Correlation heatmap saved to {os.path.join(output_dir, 'Syngas Yield.png')}")

SYNGAS + PYROLYSIS OIL + CHAR (UA+OC) 

In [ ]:


def preprocess_data(data):
    # Encode 'Sample' column if present
    if 'Sample' in data.columns:
        data['Sample'] = data['Sample'].astype('category').cat.codes
    return data

def get_model():
    return xgb.XGBRegressor(
        objective="reg:squarederror",
        n_estimators=2000,
        max_depth=3,
        learning_rate=0.012,
        subsample=0.75,
        colsample_bytree=0.13,
        reg_alpha=0.65,
        reg_lambda=0.85,
        random_state=27
    )

def train_xgboost_multi(X, y, model_name="Model"):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=27)

    model = MultiOutputRegressor(get_model())
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_r2 = r2_score(y_train, y_train_pred, multioutput='uniform_average')
    test_r2 = r2_score(y_test, y_test_pred, multioutput='uniform_average')

    print(f"\n{model_name} (XGBoost)")
    print(f"Training Set R² Score: {train_r2:.4f}")
    print(f"Test Set R² Score: {test_r2:.4f}")

    return model

# --- Main Execution ---
# Ensure your dataframe `data` is defined before this
data = preprocess_data(data)

# Define targets and feature sets
targets = ['Syngas yield (%)', 'Char yield (%)','Pyrolysis oil yield (%)']
feature_sets = {
    "Ultimate Analysis + Operating Conditions": ['C (wt%)', 'H (wt%)', 'N (wt%)', 'O (wt%)', 'S (wt%)', 'Sample (g)', 'Temperature (°C)', 'Reaction time (min)', 'Heating rate (°Cmin-1)'],
    
}

# Train and store models
trained_models = {}
for name, feature_cols in feature_sets.items():
    X, y = data[feature_cols], data[targets]
    model = train_xgboost_multi(X, y, model_name=name)
    trained_models[name] = model
    
    with open(f"{name.replace(' ', '_').lower()}_xgboost_model.pkl", 'wb') as f:
        pickle.dump(model, f)


In [ ]:
import os
import shap
import matplotlib.pyplot as plt

def shap_analysis(model, X, target_names, model_name="Model", output_dir="shap_plots"):
    print(f"\n🔍 SHAP Analysis for {model_name}")
    
    os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists
    
    for i, target in enumerate(target_names):
        print(f"\n📊 SHAP Feature Importance for Target: {target}")

        # Extract the individual estimator for this target (for multi-output models)
        xgb_model = model.estimators_[i]

        # Create SHAP explainer
        explainer = shap.Explainer(xgb_model, X)
        shap_values = explainer(X)

        # Plot 1: SHAP Summary Dot Plot
        print("🔹 SHAP Summary Dot Plot:")
        plt.figure()
        shap.summary_plot(shap_values, X, plot_type="dot", show=False)
        plt.title(f"SHAP Dot Plot - {model_name} - {target}")
        dot_plot_path = os.path.join(output_dir, f"{model_name}_{target}_shap_dot.png")
        plt.savefig(dot_plot_path, bbox_inches='tight')
        plt.close()

        # Plot 2: SHAP Summary Bar Plot
        print("🔹 SHAP Summary Bar Plot:")
        plt.figure()
        shap.summary_plot(shap_values, X, plot_type="bar", show=False)
        plt.title(f"SHAP Bar Plot - {model_name} - {target}")
        bar_plot_path = os.path.join(output_dir, f"{model_name}_{target}_shap_bar.png")
        plt.savefig(bar_plot_path, bbox_inches='tight')
        plt.close()

        print(f"✅ Saved plots to:\n  - {dot_plot_path}\n  - {bar_plot_path}")

X = data[feature_sets["Ultimate Analysis + Operating Conditions"]]
shap_analysis(trained_models["Ultimate Analysis + Operating Conditions"], X, targets, model_name="Ultimate Analysis + Operating Conditions")


SYNGAS + PYROLYSIS OIL + CHAR (PA+OC) 

In [ ]:

def preprocess_data(data):
    # Encode 'Sample' column if present
    if 'Sample' in data.columns:
        data['Sample'] = data['Sample'].astype('category').cat.codes
    return data

def get_model():
    return xgb.XGBRegressor(
        objective="reg:squarederror",
        n_estimators=900,
        max_depth=4,
        learning_rate=0.12,
        subsample=0.3,
        colsample_bytree=0.285,
        reg_alpha=0.7,
        reg_lambda=0.7,
        random_state=29
    )

def train_xgboost_multi(X, y, model_name="Model"):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=27)

    model = MultiOutputRegressor(get_model())
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_r2 = r2_score(y_train, y_train_pred, multioutput='uniform_average')
    test_r2 = r2_score(y_test, y_test_pred, multioutput='uniform_average')

    print(f"\n{model_name} (XGBoost)")
    print(f"Training Set R² Score: {train_r2:.4f}")
    print(f"Test Set R² Score: {test_r2:.4f}")

    return model

# --- Main Execution ---
# Ensure your dataframe `data` is defined before this
data = preprocess_data(data)

# Define targets and feature sets
targets = ['Syngas yield (%)', 'Char yield (%)','Pyrolysis oil yield (%)']
feature_sets = {
    
    "Proximate Analysis + Operating Conditions": ['Volatile (wt%)', 'Fixed carbon (wt%)', 'Ash (wt%)', 'Sample (g)', 'Temperature (°C)', 'Reaction time (min)', 'Heating rate (°Cmin-1)'],
    
}

# Train and store models
trained_models = {}
for name, feature_cols in feature_sets.items():
    X, y = data[feature_cols], data[targets]
    model = train_xgboost_multi(X, y, model_name=name)
    trained_models[name] = model
    
    with open(f"{name.replace(' ', '_').lower()}_xgboost_model.pkl", 'wb') as f:
        pickle.dump(model, f)


In [ ]:
import os
import shap
import matplotlib.pyplot as plt

def shap_analysis(model, X, target_names, model_name="Model", output_dir="shap_plots"):
    print(f"\n🔍 SHAP Analysis for {model_name}")
    
    os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists
    
    for i, target in enumerate(target_names):
        print(f"\n📊 SHAP Feature Importance for Target: {target}")

        # Extract the individual estimator for this target (for multi-output models)
        xgb_model = model.estimators_[i]

        # Create SHAP explainer
        explainer = shap.Explainer(xgb_model, X)
        shap_values = explainer(X)

        # Plot 1: SHAP Summary Dot Plot
        print("🔹 SHAP Summary Dot Plot:")
        plt.figure()
        shap.summary_plot(shap_values, X, plot_type="dot", show=False)
        plt.title(f"SHAP Dot Plot - {model_name} - {target}")
        dot_plot_path = os.path.join(output_dir, f"{model_name}_{target}_shap_dot.png")
        plt.savefig(dot_plot_path, bbox_inches='tight')
        plt.close()

        # Plot 2: SHAP Summary Bar Plot
        print("🔹 SHAP Summary Bar Plot:")
        plt.figure()
        shap.summary_plot(shap_values, X, plot_type="bar", show=False)
        plt.title(f"SHAP Bar Plot - {model_name} - {target}")
        bar_plot_path = os.path.join(output_dir, f"{model_name}_{target}_shap_bar.png")
        plt.savefig(bar_plot_path, bbox_inches='tight')
        plt.close()

        print(f"✅ Saved plots to:\n  - {dot_plot_path}\n  - {bar_plot_path}")
        
# Run SHAP for "Ultimate Analysis"
X = data[feature_sets["Proximate Analysis + Operating Conditions"]]
shap_analysis(trained_models["Proximate Analysis + Operating Conditions"], X, targets, model_name="Proximate Analysis + Operating Conditions")


SYNGAS + PYROLYSIS OIL + CHAR (UA+PA+OC) 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score
import xgboost as xgb

def preprocess_data(data):
    # Encode 'Sample' column if present
    if 'Sample' in data.columns:
        data['Sample'] = data['Sample'].astype('category').cat.codes
    return data

def get_model():
    return xgb.XGBRegressor(
        objective="reg:squarederror",
        n_estimators=900,
        max_depth=3,
        learning_rate=0.02,
        subsample=0.8,
        colsample_bytree=0.1,
        reg_alpha=0.1,
        reg_lambda=0.1,
        random_state=27
    )

def train_xgboost_multi(X, y, model_name="Model"):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=27)

    model = MultiOutputRegressor(get_model())
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_r2 = r2_score(y_train, y_train_pred, multioutput='uniform_average')
    test_r2 = r2_score(y_test, y_test_pred, multioutput='uniform_average')

    print(f"\n{model_name} (XGBoost)")
    print(f"Training Set R² Score: {train_r2:.4f}")
    print(f"Test Set R² Score: {test_r2:.4f}")

    return model

# --- Main Execution ---
# Ensure your dataframe `data` is defined before this
data = preprocess_data(data)

# Define targets and feature sets
targets = ['Syngas yield (%)', 'Char yield (%)','Pyrolysis oil yield (%)']
feature_sets = {
    "Ultimate + Proximate Analysis + Operating Conditions": ['C (wt%)', 'H (wt%)', 'N (wt%)', 'O (wt%)', 'S (wt%)', 'Volatile (wt%)', 'Fixed carbon (wt%)', 'Ash (wt%)', 'Sample (g)', 'Temperature (°C)', 'Reaction time (min)', 'Heating rate (°Cmin-1)']
}

# Train and store models
trained_models = {}
for name, feature_cols in feature_sets.items():
    X, y = data[feature_cols], data[targets]
    model = train_xgboost_multi(X, y, model_name=name)
    trained_models[name] = model

    # Save model to .pkl
    with open(f"{name.replace(' ', '_').lower()}_xgboost_model.pkl", 'wb') as f:
        pickle.dump(model, f)

In [ ]:
import os
import shap
import matplotlib.pyplot as plt

def shap_analysis(model, X, target_names, model_name="Model", output_dir="shap_plots"):
    print(f"\n🔍 SHAP Analysis for {model_name}")
    
    os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists
    
    for i, target in enumerate(target_names):
        print(f"\n📊 SHAP Feature Importance for Target: {target}")

        # Extract the individual estimator for this target (for multi-output models)
        xgb_model = model.estimators_[i]

        # Create SHAP explainer
        explainer = shap.Explainer(xgb_model, X)
        shap_values = explainer(X)

        # Plot 1: SHAP Summary Dot Plot
        print("🔹 SHAP Summary Dot Plot:")
        plt.figure()
        shap.summary_plot(shap_values, X, plot_type="dot", show=False)
        plt.title(f"SHAP Dot Plot - {model_name} - {target}")
        dot_plot_path = os.path.join(output_dir, f"{model_name}_{target}_shap_dot.png")
        plt.savefig(dot_plot_path, bbox_inches='tight')
        plt.close()

        # Plot 2: SHAP Summary Bar Plot
        print("🔹 SHAP Summary Bar Plot:")
        plt.figure()
        shap.summary_plot(shap_values, X, plot_type="bar", show=False)
        plt.title(f"SHAP Bar Plot - {model_name} - {target}")
        bar_plot_path = os.path.join(output_dir, f"{model_name}_{target}_shap_bar.png")
        plt.savefig(bar_plot_path, bbox_inches='tight')
        plt.close()

        print(f"✅ Saved plots to:\n  - {dot_plot_path}\n  - {bar_plot_path}")

        
# Run SHAP for "Ultimate Analysis"
X = data[feature_sets["Ultimate + Proximate Analysis + Operating Conditions"]]
shap_analysis(trained_models["Ultimate + Proximate Analysis + Operating Conditions"], X, targets, model_name="Ultimate + Proximate Analysis + Operating Conditions")


CH4

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import xgboost as xgb

def train_and_evaluate(X, y, model_type="XGBoost", model_name="Model", n_neighbors=5):
    # Split the dataset into training and testing sets (80-20 split)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=27)

    # Initialize the model based on the model_type parameter
    if model_type == "DecisionTree":
        regressor = DecisionTreeRegressor(random_state=42)
    elif model_type == "AdaBoost":
        regressor = AdaBoostRegressor(estimator=DecisionTreeRegressor(random_state=42), random_state=42)
    elif model_type == "XGBoost":
        regressor = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
    elif model_type == "GradientBoosting":
        regressor = GradientBoostingRegressor(random_state=42)
    elif model_type == "KNN":
        regressor = KNeighborsRegressor(n_neighbors=n_neighbors)
    elif model_type == "RandomForest":
        regressor = RandomForestRegressor(random_state=42)
    else:
        raise ValueError(f"Unknown model_type: {model_type}")

    # Train the model
    regressor.fit(X_train, y_train)

    # Make predictions on the training and test sets
    y_train_pred = regressor.predict(X_train)
    y_test_pred = regressor.predict(X_test)

    # Calculate performance metrics
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)

    # Print evaluation metrics
    print(f"\n{model_name} ({model_type}):")
    # print(f"Training Set Mean Squared Error: {train_mse}")
    # print(f"Training Set Mean Absolute Error: {train_mae}")
    print(f"Training Set R² Score: {train_r2}")
    # print(f"Test Set Mean Squared Error: {test_mse}")
    # print(f"Test Set Mean Absolute Error: {test_mae}")
    print(f"Test Set R² Score: {test_r2}")



# Define target variables
targets = {
    'CH4 (mol%)': 'Y_CH'
}
features = {
    "Ultimate Analysis": ['C (wt%)', 'H (wt%)', 'N (wt%)', 'O (wt%)', 'S (wt%)','Sample (g)', 'Temperature (°C)', 'Reaction time (min)','Heating rate (°Cmin-1)'],
    "Proximate Analysis": ['Volatile (wt%)', 'Fixed carbon (wt%)', 'Ash (wt%)','Sample (g)', 'Temperature (°C)', 'Reaction time (min)','Heating rate (°Cmin-1)'],
    "Ultimate + Proximate Analysis + Operating Conditions": ['C (wt%)', 'H (wt%)', 'N (wt%)', 'O (wt%)', 'S (wt%)', 'Volatile (wt%)', 'Fixed carbon (wt%)', 'Ash (wt%)','Sample (g)', 'Temperature (°C)', 'Reaction time (min)','Heating rate (°Cmin-1)']
}

# List of model types
model_types = ["DecisionTree", "AdaBoost", "XGBoost", "GradientBoosting", "KNN", "RandomForest"]

# Iterate over targets, features, and model types
for target, target_name in targets.items():
    y = data[target]
    for feature_set_name, features_list in features.items():
        print(features_list)
        X = data[features_list]
        train_and_evaluate(X, y, model_type="XGBoost", model_name=feature_set_name)



CH4 (UA+PA+OC)

In [ ]:
def preprocess_data(data):
    if 'Sample' in data.columns:
        data['Sample'] = data['Sample'].astype('category').cat.codes
    return data

# -------------------- Model Config --------------------
def get_model():
    return xgb.XGBRegressor(
        objective="reg:squarederror",
        n_estimators=400,
        max_depth=5,
        learning_rate=0.13,
        subsample=0.5,
        colsample_bytree=0.19,
        reg_alpha=0.5,
        reg_lambda=0.4,
        random_state=27
    )

# -------------------- Training and Combined Parity Plot --------------------
def train_xgboost_multi(X, y, model_name="Model"):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=27)

    model = MultiOutputRegressor(get_model())
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_r2 = r2_score(y_train, y_train_pred, multioutput='uniform_average')
    test_r2 = r2_score(y_test, y_test_pred, multioutput='uniform_average')

    print(f"\n{model_name} (XGBoost)")
    print(f"Training Set R² Score: {train_r2:.4f}")
    print(f"Test Set R² Score: {test_r2:.4f}")

    # ----- Combined Parity Plot -----
    target = y.columns[0]
    plt.figure(figsize=(6, 6))
    plt.scatter(y_train, y_train_pred, label=f"Train (R² = {train_r2:.3f})", color="Red", alpha=0.6, edgecolor="k")
    plt.scatter(y_test, y_test_pred, label=f"Test (R² = {test_r2:.3f})", color="Blue", alpha=0.6, edgecolor="k")
    plt.plot([y.min().values[0], y.max().values[0]], [y.min().values[0], y.max().values[0]], 'k--', lw=2)
    plt.xlabel(f'Actual {target}')
    plt.ylabel(f'Predicted {target}')
    plt.title(f'{model_name} - Parity Plot for {target}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return model

# -------------------- Main Execution --------------------
# Make sure your DataFrame `data` is defined
data = preprocess_data(data)

targets = ['CH4 (mol%)']
feature_sets = {
    "Ultimate + Proximate Analysis + Operating Conditions": [
        'C (wt%)', 'H (wt%)', 'N (wt%)', 'O (wt%)', 'S (wt%)',
        'Volatile (wt%)', 'Fixed carbon (wt%)', 'Ash (wt%)', 'Sample (g)',
        'Temperature (°C)', 'Reaction time (min)', 'Heating rate (°Cmin-1)'
    ]
}

trained_models = {}
for name, feature_cols in feature_sets.items():
    X = data[feature_cols]
    y = data[targets]
    model = train_xgboost_multi(X, y, model_name=name)
    trained_models[name] = model

    # --- Save model to .pkl file ---
    file_name = f"{name.replace(' ', '_').lower()}ch4_xgboost_model.pkl"
    with open(file_name, 'wb') as f:
        pickle.dump(model, f)

In [ ]:
import os
import shap
import matplotlib.pyplot as plt

def shap_analysis(model, X, target_names, model_name="Model", output_dir="SHAP_plots_CH4", save_bar_plot=False):
    print(f"\n🔍 SHAP Analysis for {model_name}")
    
    os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists

    for i, target in enumerate(target_names):
        print(f"\n📊 SHAP Feature Importance for Target: {target}")

        # Extract individual estimator (for multi-output model)
        xgb_model = model.estimators_[i]

        # SHAP Explainer
        explainer = shap.Explainer(xgb_model, X)
        shap_values = explainer(X)

        # Plot 1: SHAP Summary Dot Plot
        print("🔹 SHAP Summary Dot Plot:")
        plt.figure()
        shap.summary_plot(shap_values, X, plot_type="dot", show=False)
        plt.title(f"SHAP Dot Plot - {model_name} - {target}")
        dot_path = os.path.join(output_dir, f"{model_name}_{target}_dot.png")
        plt.savefig(dot_path, bbox_inches='tight')
        plt.close()
        print(f"✅ Dot plot saved to: {dot_path}")

        
        print("🔹 SHAP Summary Bar Plot:")
        plt.figure()
        shap.summary_plot(shap_values, X, plot_type="bar", show=False)
        plt.title(f"SHAP Bar Plot - {model_name} - {target}")
        bar_path = os.path.join(output_dir, f"{model_name}_{target}_bar.png")
        plt.savefig(bar_path, bbox_inches='tight')
        plt.close()
        print(f"✅ Bar plot saved to: {bar_path}")

        
# Run SHAP for "Ultimate Analysis"
X = data[feature_sets["Ultimate + Proximate Analysis + Operating Conditions"]]
shap_analysis(trained_models["Ultimate + Proximate Analysis + Operating Conditions"], X, targets, model_name="Ultimate + Proximate Analysis + Operating Conditions")


CH4 (PA+OC)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score
import xgboost as xgb

def preprocess_data(data):
    # Encode 'Sample' column if present
    if 'Sample' in data.columns:
        data['Sample'] = data['Sample'].astype('category').cat.codes
    return data

def get_model():
    return xgb.XGBRegressor(
        objective="reg:squarederror",
        n_estimators=700,
        max_depth=3,
        learning_rate=0.1,
        subsample=0.6,
        colsample_bytree=0.3,
        reg_alpha=0.5,
        reg_lambda=0.4,
        random_state=27
    )

# -------------------- Training and Combined Parity Plot --------------------
def train_xgboost_multi(X, y, model_name="Model"):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=27)

    model = MultiOutputRegressor(get_model())
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_r2 = r2_score(y_train, y_train_pred, multioutput='uniform_average')
    test_r2 = r2_score(y_test, y_test_pred, multioutput='uniform_average')

    print(f"\n{model_name} (XGBoost)")
    print(f"Training Set R² Score: {train_r2:.4f}")
    print(f"Test Set R² Score: {test_r2:.4f}")

    # ----- Combined Parity Plot -----
    target = y.columns[0]
    plt.figure(figsize=(6, 6))
    plt.scatter(y_train, y_train_pred, label=f"Train (R² = {train_r2:.3f})", color="Red", alpha=0.6, edgecolor="k")
    plt.scatter(y_test, y_test_pred, label=f"Test (R² = {test_r2:.3f})", color="Blue", alpha=0.6, edgecolor="k")
    plt.plot([y.min().values[0], y.max().values[0]], [y.min().values[0], y.max().values[0]], 'k--', lw=2)
    plt.xlabel(f'Actual {target}')
    plt.ylabel(f'Predicted {target}')
    plt.title(f'{model_name} - Parity Plot for {target}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return model

# --- Main Execution ---
# Ensure your dataframe `data` is defined before this
data = preprocess_data(data)

# Define targets and feature sets
targets = ['CH4 (mol%)']
feature_sets = {
    "Proximate Analysis + Operating Conditions": ['Volatile (wt%)', 'Fixed carbon (wt%)', 'Ash (wt%)','Sample (g)', 'Temperature (°C)', 'Reaction time (min)','Heating rate (°Cmin-1)'],
}

# Train and store models
trained_models = {}
for name, feature_cols in feature_sets.items():
    X = data[feature_cols]
    y = data[targets]
    model = train_xgboost_multi(X, y, model_name=name)
    trained_models[name] = model

    # --- Save model to .pkl file ---
    file_name = f"{name.replace(' ', '_').lower()}_ch4_xgboost_model.pkl"
    with open(file_name, 'wb') as f:
        pickle.dump(model, f)

In [ ]:
import os
import shap
import matplotlib.pyplot as plt

def shap_analysis(model, X, target_names, model_name="Model", output_dir="SHAP_plots_CH4", save_bar_plot=False):
    print(f"\n🔍 SHAP Analysis for {model_name}")
    
    os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists

    for i, target in enumerate(target_names):
        print(f"\n📊 SHAP Feature Importance for Target: {target}")

        # Extract individual estimator (for multi-output model)
        xgb_model = model.estimators_[i]

        # SHAP Explainer
        explainer = shap.Explainer(xgb_model, X)
        shap_values = explainer(X)

        # Plot 1: SHAP Summary Dot Plot
        print("🔹 SHAP Summary Dot Plot:")
        plt.figure()
        shap.summary_plot(shap_values, X, plot_type="dot", show=False)
        plt.title(f"SHAP Dot Plot - {model_name} - {target}")
        dot_path = os.path.join(output_dir, f"{model_name}_{target}_dot.png")
        plt.savefig(dot_path, bbox_inches='tight')
        plt.close()
        print(f"✅ Dot plot saved to: {dot_path}")

        print("🔹 SHAP Summary Bar Plot:")
        plt.figure()
        shap.summary_plot(shap_values, X, plot_type="bar", show=False)
        plt.title(f"SHAP Bar Plot - {model_name} - {target}")
        bar_path = os.path.join(output_dir, f"{model_name}_{target}_bar.png")
        plt.savefig(bar_path, bbox_inches='tight')
        plt.close()
        print(f"✅ Bar plot saved to: {bar_path}")

        
# Run SHAP for "Ultimate Analysis"
X = data[feature_sets["Proximate Analysis + Operating Conditions"]]
shap_analysis(trained_models["Proximate Analysis + Operating Conditions"], X, targets, model_name="Proximate Analysis + Operating Conditions")


CH4 (UA+OC)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score
import xgboost as xgb

def preprocess_data(data):
    # Encode 'Sample' column if present
    if 'Sample' in data.columns:
        data['Sample'] = data['Sample'].astype('category').cat.codes
    return data

def get_model():
    return xgb.XGBRegressor(
        objective="reg:squarederror",
        n_estimators=900,
        max_depth=3,
        learning_rate=0.1,
        subsample=0.5,
        colsample_bytree=0.3,
        reg_alpha=0.5,
        reg_lambda=0.4,
        random_state=27
    )

# -------------------- Training and Combined Parity Plot --------------------
def train_xgboost_multi(X, y, model_name="Model"):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=27)

    model = MultiOutputRegressor(get_model())
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_r2 = r2_score(y_train, y_train_pred, multioutput='uniform_average')
    test_r2 = r2_score(y_test, y_test_pred, multioutput='uniform_average')

    print(f"\n{model_name} (XGBoost)")
    print(f"Training Set R² Score: {train_r2:.4f}")
    print(f"Test Set R² Score: {test_r2:.4f}")

    # ----- Combined Parity Plot -----
    target = y.columns[0]
    plt.figure(figsize=(6, 6))
    plt.scatter(y_train, y_train_pred, label=f"Train (R² = {train_r2:.3f})", color="Red", alpha=0.6, edgecolor="k")
    plt.scatter(y_test, y_test_pred, label=f"Test (R² = {test_r2:.3f})", color="Blue", alpha=0.6, edgecolor="k")
    plt.plot([y.min().values[0], y.max().values[0]], [y.min().values[0], y.max().values[0]], 'k--', lw=2)
    plt.xlabel(f'Actual {target}')
    plt.ylabel(f'Predicted {target}')
    plt.title(f'{model_name} - Parity Plot for {target}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return model

# --- Main Execution ---
# Ensure your dataframe `data` is defined before this
data = preprocess_data(data)

# Define targets and feature sets
targets = ['CH4 (mol%)']
feature_sets = {
    "Ultimate Analysis + Operating Conditions": ['C (wt%)', 'H (wt%)', 'N (wt%)', 'O (wt%)', 'S (wt%)','Sample (g)', 'Temperature (°C)', 'Reaction time (min)','Heating rate (°Cmin-1)'],
}

# Train and store models
for name, feature_cols in feature_sets.items():
    X = data[feature_cols]
    y = data[targets]
    model = train_xgboost_multi(X, y, model_name=name)
    trained_models[name] = model

    # --- Save model to .pkl file ---
    file_name = f"{name.replace(' ', '_').lower()}_ch4_xgboost_model.pkl"
    with open(file_name, 'wb') as f:
        pickle.dump(model, f)

In [ ]:
import os
import shap
import matplotlib.pyplot as plt

def shap_analysis(model, X, target_names, model_name="Model", output_dir="SHAP_plots_CH4", save_bar_plot=False):
    print(f"\n🔍 SHAP Analysis for {model_name}")
    
    os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists

    for i, target in enumerate(target_names):
        print(f"\n📊 SHAP Feature Importance for Target: {target}")

        # Extract individual estimator (for multi-output model)
        xgb_model = model.estimators_[i]

        # SHAP Explainer
        explainer = shap.Explainer(xgb_model, X)
        shap_values = explainer(X)

        # Plot 1: SHAP Summary Dot Plot
        print("🔹 SHAP Summary Dot Plot:")
        plt.figure()
        shap.summary_plot(shap_values, X, plot_type="dot", show=False)
        plt.title(f"SHAP Dot Plot - {model_name} - {target}")
        dot_path = os.path.join(output_dir, f"{model_name}_{target}_dot.png")
        plt.savefig(dot_path, bbox_inches='tight')
        plt.close()
        print(f"✅ Dot plot saved to: {dot_path}")

        print("🔹 SHAP Summary Bar Plot:")
        plt.figure()
        shap.summary_plot(shap_values, X, plot_type="bar", show=False)
        plt.title(f"SHAP Bar Plot - {model_name} - {target}")
        bar_path = os.path.join(output_dir, f"{model_name}_{target}_bar.png")
        plt.savefig(bar_path, bbox_inches='tight')
        plt.close()
        print(f"✅ Bar plot saved to: {bar_path}")

        
# Run SHAP for "Ultimate Analysis"
X = data[feature_sets["Ultimate Analysis + Operating Conditions"]]
shap_analysis(trained_models["Ultimate Analysis + Operating Conditions"], X, targets, model_name="Ultimate Analysis + Operating Conditions")


H2 (UA+OC)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score
import xgboost as xgb

def preprocess_data(data):
    # Encode 'Sample' column if present
    if 'Sample' in data.columns:
        data['Sample'] = data['Sample'].astype('category').cat.codes
    return data

def get_model():
    return xgb.XGBRegressor(
        objective="reg:squarederror",
        n_estimators=1100,
        max_depth=3,
        learning_rate=0.0199,
        subsample=0.1,
        colsample_bytree=0.9,
        reg_alpha=0.05,
        reg_lambda=0.8,
        random_state=27
    )

# -------------------- Training and Combined Parity Plot --------------------
def train_xgboost_multi(X, y, model_name="Model"):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=27)

    model = MultiOutputRegressor(get_model())
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_r2 = r2_score(y_train, y_train_pred, multioutput='uniform_average')
    test_r2 = r2_score(y_test, y_test_pred, multioutput='uniform_average')

    print(f"\n{model_name} (XGBoost)")
    print(f"Training Set R² Score: {train_r2:.4f}")
    print(f"Test Set R² Score: {test_r2:.4f}")

    # ----- Combined Parity Plot -----
    target = y.columns[0]
    plt.figure(figsize=(6, 6))
    plt.scatter(y_train, y_train_pred, label=f"Train (R² = {train_r2:.3f})", color="Red", alpha=0.6, edgecolor="k")
    plt.scatter(y_test, y_test_pred, label=f"Test (R² = {test_r2:.3f})", color="Blue", alpha=0.6, edgecolor="k")
    plt.plot([y.min().values[0], y.max().values[0]], [y.min().values[0], y.max().values[0]], 'k--', lw=2)
    plt.xlabel(f'Actual {target}')
    plt.ylabel(f'Predicted {target}')
    plt.title(f'{model_name} - Parity Plot for {target}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return model

# --- Main Execution ---
# Ensure your dataframe `data` is defined before this
data = preprocess_data(data)

# Define targets and feature sets
targets = ['H2 (mol%)']
feature_sets = {
    "Ultimate Analysis + Operating Conditions": ['C (wt%)', 'H (wt%)', 'N (wt%)', 'O (wt%)', 'S (wt%)','Sample (g)', 'Temperature (°C)', 'Reaction time (min)','Heating rate (°Cmin-1)'],
}

# Train and store models
trained_models = {}
for name, feature_cols in feature_sets.items():
    X = data[feature_cols]
    y = data[targets]
    model = train_xgboost_multi(X, y, model_name=name)
    trained_models[name] = model

    # --- Save model to .pkl file ---
    file_name = f"{name.replace(' ', '_').lower()}_h2_xgboost_model.pkl"
    with open(file_name, 'wb') as f:
        pickle.dump(model, f)

In [ ]:
import os
import shap
import matplotlib.pyplot as plt

def shap_analysis(model, X, target_names, model_name="Model", output_dir="SHAP_plots_H2", save_bar_plot=False):
    print(f"\n🔍 SHAP Analysis for {model_name}")
    
    os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists

    for i, target in enumerate(target_names):
        print(f"\n📊 SHAP Feature Importance for Target: {target}")

        # Extract individual estimator (for multi-output model)
        xgb_model = model.estimators_[i]

        # SHAP Explainer
        explainer = shap.Explainer(xgb_model, X)
        shap_values = explainer(X)

        # Plot 1: SHAP Summary Dot Plot
        print("🔹 SHAP Summary Dot Plot:")
        plt.figure()
        shap.summary_plot(shap_values, X, plot_type="dot", show=False)
        plt.title(f"SHAP Dot Plot - {model_name} - {target}")
        dot_path = os.path.join(output_dir, f"{model_name}_{target}_dot.png")
        plt.savefig(dot_path, bbox_inches='tight')
        plt.close()
        print(f"✅ Dot plot saved to: {dot_path}")

        print("🔹 SHAP Summary Bar Plot:")
        plt.figure()
        shap.summary_plot(shap_values, X, plot_type="bar", show=False)
        plt.title(f"SHAP Bar Plot - {model_name} - {target}")
        bar_path = os.path.join(output_dir, f"{model_name}_{target}_bar.png")
        plt.savefig(bar_path, bbox_inches='tight')
        plt.close()
        print(f"✅ Bar plot saved to: {bar_path}")

            
# Run SHAP for "Ultimate Analysis"
X = data[feature_sets["Ultimate Analysis + Operating Conditions"]]
shap_analysis(trained_models["Ultimate Analysis + Operating Conditions"], X, targets, model_name="Ultimate Analysis + Operating Conditions")


H2 (PA+OC)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score
import xgboost as xgb

def preprocess_data(data):
    # Encode 'Sample' column if present
    if 'Sample' in data.columns:
        data['Sample'] = data['Sample'].astype('category').cat.codes
    return data

def get_model():
    return xgb.XGBRegressor(
        objective="reg:squarederror",
        n_estimators=1100,
        max_depth=3,
        learning_rate=0.04,
        subsample=0.1,
        colsample_bytree=0.9,
        reg_alpha=0.05,
        reg_lambda=0.8,
        random_state=27
    )

# -------------------- Training and Combined Parity Plot --------------------
def train_xgboost_multi(X, y, model_name="Model"):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=27)

    model = MultiOutputRegressor(get_model())
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_r2 = r2_score(y_train, y_train_pred, multioutput='uniform_average')
    test_r2 = r2_score(y_test, y_test_pred, multioutput='uniform_average')

    print(f"\n{model_name} (XGBoost)")
    print(f"Training Set R² Score: {train_r2:.4f}")
    print(f"Test Set R² Score: {test_r2:.4f}")

    # ----- Combined Parity Plot -----
    target = y.columns[0]
    plt.figure(figsize=(6, 6))
    plt.scatter(y_train, y_train_pred, label=f"Train (R² = {train_r2:.3f})", color="Red", alpha=0.6, edgecolor="k")
    plt.scatter(y_test, y_test_pred, label=f"Test (R² = {test_r2:.3f})", color="Blue", alpha=0.6, edgecolor="k")
    plt.plot([y.min().values[0], y.max().values[0]], [y.min().values[0], y.max().values[0]], 'k--', lw=2)
    plt.xlabel(f'Actual {target}')
    plt.ylabel(f'Predicted {target}')
    plt.title(f'{model_name} - Parity Plot for {target}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return model

# --- Main Execution ---
# Ensure your dataframe `data` is defined before this
data = preprocess_data(data)

# Define targets and feature sets
targets = ['H2 (mol%)']
feature_sets = {
    "Proximate Analysis + Operating Conditions": ['Volatile (wt%)', 'Fixed carbon (wt%)', 'Ash (wt%)','Sample (g)', 'Temperature (°C)', 'Reaction time (min)','Heating rate (°Cmin-1)'],
}

# Train and store models
trained_models = {}
for name, feature_cols in feature_sets.items():
    X = data[feature_cols]
    y = data[targets]
    model = train_xgboost_multi(X, y, model_name=name)
    trained_models[name] = model

    # --- Save model to .pkl file ---
    file_name = f"{name.replace(' ', '_').lower()}_h2_xgboost_model.pkl"
    with open(file_name, 'wb') as f:
        pickle.dump(model, f)

In [ ]:
def shap_analysis(model, X, target_names, model_name="Model", output_dir="SHAP_plots_H2", save_bar_plot=False):
    print(f"\n🔍 SHAP Analysis for {model_name}")
    
    os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists

    for i, target in enumerate(target_names):
        print(f"\n📊 SHAP Feature Importance for Target: {target}")

        # Extract individual estimator (for multi-output model)
        xgb_model = model.estimators_[i]

        # SHAP Explainer
        explainer = shap.Explainer(xgb_model, X)
        shap_values = explainer(X)

        # Plot 1: SHAP Summary Dot Plot
        print("🔹 SHAP Summary Dot Plot:")
        plt.figure()
        shap.summary_plot(shap_values, X, plot_type="dot", show=False)
        plt.title(f"SHAP Dot Plot - {model_name} - {target}")
        dot_path = os.path.join(output_dir, f"{model_name}_{target}_dot.png")
        plt.savefig(dot_path, bbox_inches='tight')
        plt.close()
        print(f"✅ Dot plot saved to: {dot_path}")

        print("🔹 SHAP Summary Bar Plot:")
        plt.figure()
        shap.summary_plot(shap_values, X, plot_type="bar", show=False)
        plt.title(f"SHAP Bar Plot - {model_name} - {target}")
        bar_path = os.path.join(output_dir, f"{model_name}_{target}_bar.png")
        plt.savefig(bar_path, bbox_inches='tight')
        plt.close()
        print(f"✅ Bar plot saved to: {bar_path}")

        
# Run SHAP for "Ultimate Analysis"
X = data[feature_sets["Proximate Analysis + Operating Conditions"]]
shap_analysis(trained_models["Proximate Analysis + Operating Conditions"], X, targets, model_name="Proximate Analysis + Operating Conditions")


H2 (UA+PA+OC)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score
import xgboost as xgb

def preprocess_data(data):
    # Encode 'Sample' column if present
    if 'Sample' in data.columns:
        data['Sample'] = data['Sample'].astype('category').cat.codes
    return data

def get_model():
    return xgb.XGBRegressor(
        objective="reg:squarederror",
        n_estimators=800,
        max_depth=5,
        learning_rate=0.01,
        subsample=0.2,
        colsample_bytree=0.9,
        reg_alpha=0.4,
        reg_lambda=0.9,
        random_state=27
    )

# -------------------- Training and Combined Parity Plot --------------------
def train_xgboost_multi(X, y, model_name="Model"):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=27)

    model = MultiOutputRegressor(get_model())
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_r2 = r2_score(y_train, y_train_pred, multioutput='uniform_average')
    test_r2 = r2_score(y_test, y_test_pred, multioutput='uniform_average')

    print(f"\n{model_name} (XGBoost)")
    print(f"Training Set R² Score: {train_r2:.4f}")
    print(f"Test Set R² Score: {test_r2:.4f}")

    # ----- Combined Parity Plot -----
    target = y.columns[0]
    plt.figure(figsize=(6, 6))
    plt.scatter(y_train, y_train_pred, label=f"Train (R² = {train_r2:.3f})", color="Red", alpha=0.6, edgecolor="k")
    plt.scatter(y_test, y_test_pred, label=f"Test (R² = {test_r2:.3f})", color="Blue", alpha=0.6, edgecolor="k")
    plt.plot([y.min().values[0], y.max().values[0]], [y.min().values[0], y.max().values[0]], 'k--', lw=2)
    plt.xlabel(f'Actual {target}')
    plt.ylabel(f'Predicted {target}')
    plt.title(f'{model_name} - Parity Plot for {target}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return model

# --- Main Execution ---
# Ensure your dataframe `data` is defined before this
data = preprocess_data(data)

# Define targets and feature sets
targets = ['H2 (mol%)']
feature_sets = {
    "Ultimate + Proximate Analysis + Operating Conditions": ['C (wt%)', 'H (wt%)', 'N (wt%)', 'O (wt%)', 'S (wt%)', 'Volatile (wt%)', 'Fixed carbon (wt%)', 'Ash (wt%)', 'Sample (g)', 'Temperature (°C)', 'Reaction time (min)', 'Heating rate (°Cmin-1)'],
}

# Train and store models
trained_models = {}
for name, feature_cols in feature_sets.items():
    X = data[feature_cols]
    y = data[targets]
    model = train_xgboost_multi(X, y, model_name=name)
    trained_models[name] = model

    # --- Save model to .pkl file ---
    file_name = f"{name.replace(' ', '_').lower()}_h2_xgboost_model.pkl"
    with open(file_name, 'wb') as f:
        pickle.dump(model, f)

In [ ]:
def shap_analysis(model, X, target_names, model_name="Model", output_dir="SHAP_plots_H2", save_bar_plot=False):
    print(f"\n🔍 SHAP Analysis for {model_name}")
    
    os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists

    for i, target in enumerate(target_names):
        print(f"\n📊 SHAP Feature Importance for Target: {target}")

        # Extract individual estimator (for multi-output model)
        xgb_model = model.estimators_[i]

        # SHAP Explainer
        explainer = shap.Explainer(xgb_model, X)
        shap_values = explainer(X)

        # Plot 1: SHAP Summary Dot Plot
        print("🔹 SHAP Summary Dot Plot:")
        plt.figure()
        shap.summary_plot(shap_values, X, plot_type="dot", show=False)
        plt.title(f"SHAP Dot Plot - {model_name} - {target}")
        dot_path = os.path.join(output_dir, f"{model_name}_{target}_dot.png")
        plt.savefig(dot_path, bbox_inches='tight')
        plt.close()
        print(f"✅ Dot plot saved to: {dot_path}")

        print("🔹 SHAP Summary Bar Plot:")
        plt.figure()
        shap.summary_plot(shap_values, X, plot_type="bar", show=False)
        plt.title(f"SHAP Bar Plot - {model_name} - {target}")
        bar_path = os.path.join(output_dir, f"{model_name}_{target}_bar.png")
        plt.savefig(bar_path, bbox_inches='tight')
        plt.close()
        print(f"✅ Bar plot saved to: {bar_path}")

        
# Run SHAP for "Ultimate Analysis"
X = data[feature_sets[ "Ultimate + Proximate Analysis + Operating Conditions"]]
shap_analysis(trained_models[ "Ultimate + Proximate Analysis + Operating Conditions"], X, targets, model_name= "Ultimate + Proximate Analysis + + Operating Conditions")


FRONT END


In [ ]:
from sklearn.multioutput import MultiOutputRegressor
import numpy as np
import gradio as gr
import joblib

# Paths to models
model_paths = {
    "All Parameters": "ultimate_+_proximate_analysis_+_operating_conditions_xgboost_model.pkl",
    "Proximate & Operating": "proximate_analysis_+_operating_conditions_xgboost_model.pkl",
    "UA & Operating": "ultimate_analysis_+_operating_conditions_xgboost_model.pkl",
    "All Parameters CH4": "ultimate_+_proximate_analysis_+_operating_conditionsch4_xgboost_model.pkl",
    "Proximate & Operating CH4": "proximate_analysis_+_operating_conditions_ch4_xgboost_model.pkl",
    "UA & Operating CH4": "ultimate_analysis_+_operating_conditions_ch4_xgboost_model.pkl",
    "All Parameters H2": "ultimate_+_proximate_analysis_+_operating_conditions_h2_xgboost_model.pkl",
    "Proximate & Operating H2": "proximate_analysis_+_operating_conditions_h2_xgboost_model.pkl",
    "UA & Operating H2": "ultimate_analysis_+_operating_conditions_h2_xgboost_model.pkl",
}

def load_model(path_key):
    try:
        return joblib.load(model_paths[path_key])
    except Exception as e:
        print(f"Error loading model '{path_key}': {e}")
        return None

input_features = {
    "All Parameters": ['C (wt%)', 'H (wt%)', 'N (wt%)', 'O (wt%)', 'S (wt%)', 'Volatile (wt%)', 'Fixed carbon (wt%)', 'Ash (wt%)','Sample (g)', 'Temperature (°C)', 'Reaction time (min)','Heating rate (°C/min)'],
    "PA & Operating": ['Volatile (wt%)', 'Fixed carbon (wt%)', 'Ash (wt%)','Sample (g)', 'Temperature (°C)', 'Reaction time (min)','Heating rate (°C/min)'],
    "UA & Operating": ['C (wt%)', 'H (wt%)', 'N (wt%)', 'O (wt%)', 'S (wt%)','Sample (g)', 'Temperature (°C)', 'Reaction time (min)','Heating rate (°C/min)']
}

output_features = ['Syngas yield (%)',  'Char yield (%)', 'Pyrolysis oil yield (%)', 'CH4 (mol%)','H2 (mol%)']

# Custom prediction function to combine models
def biomass_prediction(input_category, input_values):
    input_data = np.array(input_values).reshape(1, -1)
    
    if np.any(input_data < 0):
        return ["Invalid input" for _ in output_features]
    if np.all(input_data == 0):
        return ["0 %" for _ in output_features]

    result = [""] * len(output_features)

    # Predict main outputs
    if input_category == "PA & Operating":
        base_model = load_model("PA & Operating")
        ch4_model = load_model("PA & Operating CH4")
        h2_model = load_model("PA & Operating H2")
    elif input_category == "UA & Operating":
        base_model = load_model("UA & Operating")
        ch4_model = load_model("UA & Operating CH4")
        h2_model = load_model("UA & Operating H2")
    elif input_category == "All Parameters":
        base_model = load_model("All Parameters")
        ch4_model = load_model("All Parameters CH4")
        h2_model = load_model("All Parameters H2")
    else:
        return ["Invalid category"] * len(output_features)

    try:
        base_pred = base_model.predict(input_data)[0]
        h2_pred = h2_model.predict(input_data)[0]
        ch4_pred = ch4_model.predict(input_data)[0]
        
        # Assign predictions
        result[0] = f"{round(base_pred[0], 2)} %"
        result[1] = f"{round(base_pred[1], 2)} %"
        result[2] = f"{round(base_pred[2], 2)} %"
        result[3] = f"{round(ch4_pred[0], 2)} %"
        result[4] = f"{round(h2_pred[0], 2)} %"
        
    except Exception as e:
        print("Prediction error:", e)
        result = ["Error"] * len(output_features)

    return result

# Build GUI
def create_gui():
    with gr.Blocks(title="Coal Pyrolysis Prediction", theme=gr.themes.Monochrome()) as interface:
        gr.Markdown("# Coal Pyrolysis Prediction Tool")
        gr.Markdown(
            "This tool predicts pyrolysis product yields from coal. Choose the input type and provide values."
        )

        with gr.Tabs() as tabs:
            for tab_name in ["All Parameters", "PA & Operating", "UA & Operating"]:
                with gr.TabItem(tab_name):
                    input_boxes = []
                    with gr.Row():
                        with gr.Column():
                            for i, feat in enumerate(input_features[tab_name]):
                                if i < len(input_features[tab_name]) // 2 + 1:
                                    input_boxes.append(gr.Number(label=feat, value=0))
                        with gr.Column():
                            for i, feat in enumerate(input_features[tab_name]):
                                if i >= len(input_features[tab_name]) // 2 + 1:
                                    input_boxes.append(gr.Number(label=feat, value=0))

                    btn = gr.Button(f"Predict", variant="primary")
                    outputs = [gr.Textbox(label=feat) for feat in output_features]

                    btn.click(
                        fn=lambda *inputs, cat=tab_name: biomass_prediction(cat, inputs),
                        inputs=input_boxes,
                        outputs=outputs
                    )
    
    interface.launch(inline=False)

if __name__ == '__main__':
    create_gui()

